In [ ]:
from refactor.generator.model import RetrievalAugmentedGenerator
from refactor.dpo.model import DPOTrainModule

ckpt_path = '../dpo.ckpt'

In [ ]:
# tac_gen = DPOTrainModule.load(
#     ckpt_path, device='cuda', freeze=False
# )

In [ ]:
tac_gen = RetrievalAugmentedGenerator.load(
    ckpt_path, device='cuda', freeze=True
)


In [ ]:
tac_gen.batch_generate([trace.tree])

In [ ]:
def gen(state):
    tokenized_state = tac_gen.tokenizer(
    state,
    padding="longest",
    max_length=2300,
    truncation=True,
    return_tensors="pt",
    )

    state_ids = tokenized_state.input_ids.to('cuda')
    state_mask = tokenized_state.attention_mask.to('cuda')

    # )
    #
    output = tac_gen.generator.generate(
    input_ids=state_ids,
    attention_mask=state_mask,
    max_length=2300,
    length_penalty=0.0,
    do_sample=True,
    num_return_sequences=8,
    output_scores=True,
    return_dict_in_generate=True,
    top_p=0.95,
    )


    return output


In [ ]:
import torch

output = gen(trace.tree.goal)




In [ ]:
transitions = tac_gen.generator.compute_transition_scores(output.sequences, output.scores, normalize_logits=True)

In [ ]:
transitions.shape

In [ ]:
transitions[transitions != -torch.inf]

In [ ]:
torch.sum(transitions[3][transitions[3] != -torch.inf])

In [ ]:
tac_gen.tokenizer.decode(output.sequences[7])

In [ ]:
transitions[0]

In [ ]:
for tok, score in zip(output.sequences[2], transitions[2]):
    print (tac_gen.tokenizer.decode(tok),score)

In [ ]:
tac_gen.tokenizer.

In [ ]:
print (torch.sum(torch.nonzero(output.sequences[0])[0]))

In [ ]:
output.sequences.shape

In [ ]:
len(output.scores)

In [ ]:
scores = torch.stack(output.scores, dim=1)

In [ ]:
scores.shape

In [ ]:
scores[0]

In [ ]:
scores[0][scores[0] > -torch.inf]

In [ ]:
torch.gather(scores[0], 1,

In [ ]:
(scores[0] > -torch.inf)

In [ ]:
torch.nonzero(scores[0] != -torch.inf)

In [ ]:
from peft import LoraModel, LoraConfig

In [ ]:
tac_gen

In [ ]:
target_modules = ['q', 'k', 'v', 'o', 'wo', 'lm_head']
config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=16,
    target_modules=target_modules,
    lora_dropout=0.01,
)

model = tac_gen.generator
lora_model = LoraModel(model, config, "default")

In [ ]:
from peft import get_peft_model
peft_model = get_peft_model(model, config)


In [ ]:
peft_model.print_trainable_parameters()

In [ ]:
from refactor.process_traces import get_traces

trace = get_traces('../experiments/runs/eval_loop/bestfs_2023_11_16/07_59_20/traces/abs_sub_round_eq_min')[0]


In [ ]:
print (tac_gen)

In [ ]:
def reconstruct_scores(state, sequence):
    tokenized_sequence= tac_gen.tokenizer(
        sequence,
        padding="longest",
        max_length=2300,
        truncation=True,
        return_tensors="pt",
    )

    tokenized_state = tac_gen.tokenizer(
        state,
        padding="longest",
        max_length=2300,
        truncation=True,
        return_tensors="pt",
    )

    state_ids = tokenized_state.input_ids.cuda()
    state_mask = tokenized_state.attention_mask.cuda()

    targets = tokenized_sequence.input_ids.cuda()

    targets[targets == tac_gen.tokenizer.pad_token_id] = -100



    output = tac_gen.generator(
        input_ids=state_ids,
        attention_mask=state_mask,
        labels=targets
    ).logits

    return output, tokenized_sequence.input_ids.cuda()

In [ ]:
edges = trace.tree.out_edges

In [ ]:
[(e, e.distance_to_proof(), e.dst[0].status) for e in edges]

In [ ]:
trace.tree.out_edges[:2]

In [ ]:
import torch
tacs = ['rw <a>round_eq</a>']

In [ ]:
logits, seq_ids = reconstruct_scores([trace.tree.goal], tacs)

In [ ]:
normalised = torch.log_softmax(logits, -1)


In [ ]:
outs = torch.gather(normalised, -1, seq_ids.unsqueeze(-1)).squeeze(-1)


In [ ]:
outs[seq_ids == tac_gen.tokenizer.pad_token_id] = 0

In [ ]:
torch.sum(outs, dim=1)